In [1]:
import numpy as np
import scipy.io as spio

In [13]:
mat = spio.loadmat('MUTAG.mat', squeeze_me = True)
mutag = mat['MUTAG']
lmutag = mat['lmutag']
N = np.size(mutag)

tol = 0
#get total number of nodes in the dataset  
for i in range(N):
        graph = mutag[i]
        tol += len(graph[1].tolist()[0])

### WL Edge Kernel

In [3]:
def original_count(nl, Map, ctr): 
    num_labels = nl.shape[0]
    labels = np.zeros(num_labels) 
    
    for i in range(num_labels): 
        str_label = str(nl[i]) 
        if(str_label in Map): 
            labels[i] = Map[str_label] 
        else: 
            Map[str_label] = ctr
            labels[i] = ctr
            ctr += 1 
        
    labels = np.int64(labels)
    return {'labels': labels, 'Map': Map, 'ctr': ctr}

In [4]:
def get_edge_map(nl, am): 
    """
    nl: node labels 
    am: adjacent matrix of a node 
    """
    
    n = nl.shape[0]
    square = np.matlib.repmat(np.asmatrix(nl).T, 1, n)
    Start = np.minimum(square, square.T)
    End = np.maximum(square, square.T)
    Edges = np.triu(am, 1)
    index = np.where(Edges == 1)
    row = index[0]; 
    col = index[1];
    
    edge_map = dict() 
    n_pairs = len(row)
    for k in range(n_pairs): 
        i = row[k] 
        j = col[k]
        start_ep = Start[i, j]
        end_ep = End[i, j]

        edge = str(start_ep) + "-" + str(end_ep) #key: ordered pair name
        if (edge in edge_map): 
            edge_map[edge] += 1; 
        else: 
            edge_map[edge] = 1; 
            

    return edge_map


"""
get_edge_map(l1, am1)
#return: 
{'3-3': 24, '3-6': 1, '6-7': 2}
"""

In [5]:
def compress_label(nl, am):  
    Map = dict(); ctr = 0
    phi1 = []
    phi2 = [] 
    new_labels = np.zeros(nl.shape[0])
    for i in range(nl.shape[0]): 
        neighbors_labels = nl[np.where(am[i] == 1)]
        str_label = str(nl[i])
        
        for neighbor in range(neighbors_labels.shape[0]): 
            str_label += str(neighbor)
        
        if(str_label in Map): 
            new_labels[i] = Map[str_label]
        else: 
            Map[str_label] = ctr
            new_labels[i] = ctr
            ctr += 1 
            
    return {'new_labels': new_labels, 'Map': Map, 'ctr': ctr}        

In [12]:
"""
edge kernel of two graphs  
"""
def wl_edge_kernel (graph1, graph2, H):  
    K = 0 

    #read graph structure 
    am1, l1 = graph1[0], graph1[1]
    am2, l2 = graph2[0], graph2[1]
    l1 = l1.tolist()[0]
    l2 = l2.tolist()[0]
    
    #number of distinct nodes 
    #size1 = len(set(l1))
    #size2 = len(set(l2))
    
    
    ###Step 1: Initialization/original count
    Map = dict();
    ctr = 0 
    
    #for graph 1 
    res1 = original_count(l1, Map, ctr)
    Map = res1['Map']
    ctr = res1['ctr']
    
    #for graph 2
    res2 = original_count(l2, Map, ctr)
    Map = res2['Map']
    ctr = res2['ctr']
    
    #get initial labels  
    labels1 = res1['labels']
    labels2 = res2['labels']
    
    ###Step 2: Find ordered endpoints labels and # of occurences
    edge_map1 = get_edge_map(labels1, am1)
    edge_map2 = get_edge_map(labels2, am2)
    
    ###Step 3: Compute inner product of edge_map1 & edge_map2 
    edges1 = set(edge_map1.keys())
    edges2 = set(edge_map2.keys())
    all_edges = edges1 | edges2 
    phi1 = []
    phi2 = []
    for edge in all_edges: 
        phi1.append(edge_map1[edge] if edge in edges1 else 0)
        phi2.append(edge_map2[edge] if edge in edges2 else 0)
    
    K += np.dot(np.asmatrix(phi1), np.asmatrix(phi2).T)
    
    
    #Step 4: Repeat the above process for H iterations. 
    
    for h in range(H):

        ###Step 4.a: get compressed labels 

        #for graph 1 
        res1 = compress_label(labels1, am1)
        Map = res1['Map']
        ctr = res1['ctr']

        #for graph 2
        res2 = compress_label(labels2, am2)
        Map = res2['Map']
        ctr = res2['ctr']

        #update labels for h-th iteration
        labels1 = res1['new_labels']
        labels2 = res2['new_labels']

        ###Step 4.b: Find ordered endpoints labels of edges in two graphs
        edge_map1 = get_edge_map(labels1, am1)
        edge_map2 = get_edge_map(labels2, am2)

        ###Step 4.c: compute inner product of edge_map1 & edge_map2 
        edges1 = set(edge_map1.keys())
        edges2 = set(edge_map2.keys())
        all_edges = edges1 | edges2 
        phi1 = []
        phi2 = []
        for edge in all_edges: 
            phi1.append(edge_map1[edge] if edge in edges1 else 0)
            phi2.append(edge_map2[edge] if edge in edges2 else 0)

        K += np.dot(np.asmatrix(phi1), np.asmatrix(phi2).T)

    return K

Test Block